first we need to train a parametric-umap network for each dataset... (5 datasets x 2 dimensions)
For umap-learn, UMAP AE, Param. UMAP, PCA
- load dataset
- load network
- compute reconstruction MSE
- count time

In [1]:
# reload packages
%load_ext autoreload
%autoreload 2

### Choose GPU (this may not be needed on your computer)

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [3]:
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if len(gpu_devices)>0:
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)

In [4]:
gpu_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
import numpy as np
import pickle
import pandas as pd
import time
from umap import UMAP

In [6]:
from tfumap.umap import tfUMAP
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [7]:
from tqdm.autonotebook import tqdm

In [8]:
from tfumap.paths import ensure_dir, MODEL_DIR, DATA_DIR

In [9]:
output_dir = MODEL_DIR/'projections' 

In [10]:
reconstruction_acc_df = pd.DataFrame(columns = ['method_', 'dimensions', 'dataset', 'MSE', 'MAE', 'MedAE', 'R2'])

In [11]:
reconstruction_speed_df = pd.DataFrame(columns = ['method_', 'dimensions', 'dataset', 'embed_time', 'recon_time', 'speed', 'nex'])

### MNIST

In [12]:
dataset = 'fmnist'
dims = (28,28,1)

##### load dataset

In [13]:
from tensorflow.keras.datasets import fashion_mnist

# load dataset
(train_images, Y_train), (test_images, Y_test) = fashion_mnist.load_data()
X_train = (train_images/255.).astype('float32')
X_test = (test_images/255.).astype('float32')
X_train = X_train.reshape((len(X_train), np.product(np.shape(X_train)[1:])))
X_test = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

# subset a validation set
n_valid = 10000
X_valid = X_train[-n_valid:]
Y_valid = Y_train[-n_valid:]
X_train = X_train[:-n_valid]
Y_train = Y_train[:-n_valid]

# flatten X
X_train_flat = X_train.reshape((len(X_train), np.product(np.shape(X_train)[1:])))
X_test_flat = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
X_valid_flat= X_valid.reshape((len(X_valid), np.product(np.shape(X_valid)[1:])))
X_test = X_test.reshape((10000, 28,28,1))
print(len(X_train), len(X_valid), len(X_test))

50000 10000 10000


### AE 

##### 2 dims

In [14]:
load_loc = output_dir / dataset / 'autoencoder' 

In [15]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    decoding_method = "autoencoder",
    batch_size = 100,
    dims = dims
)

In [16]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [17]:
decoder = tf.keras.models.load_model((load_loc / 'decoder').as_posix())
embedder.decoder = decoder

In [18]:
X_recon = decoder(encoder(X_test)).numpy()
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

In [19]:
MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

In [20]:
reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['AE', 2, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,fmnist,0.024047,0.082691,0.035981,0.617169


##### 64 dims

In [21]:
load_loc = output_dir / dataset / '64' / 'autoencoder' 

In [22]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    decoding_method = "autoencoder",
    batch_size = 100,
    dims = dims
)

In [23]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [24]:
decoder = tf.keras.models.load_model((load_loc / 'decoder').as_posix())
embedder.decoder = decoder

In [25]:
X_recon = decoder(encoder(X_test)).numpy()
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

In [26]:
MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

In [27]:
reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['AE', 64, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,fmnist,0.024047,0.082691,0.035981,0.617169
1,AE,64,fmnist,0.009165,0.043426,0.013206,0.807800


### Network

##### 2 dims

In [28]:
load_loc = output_dir / dataset / 'recon-network' 

In [29]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    decoding_method = "network",
    batch_size = 100,
    dims = dims
)

In [30]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [31]:
decoder = tf.keras.models.load_model((load_loc / 'decoder').as_posix())
embedder.decoder = decoder

In [32]:
n_repeats = 10
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    z_test = encoder(X_test)
    end_time = time.monotonic()
    print("seconds: ", end_time - start_time)
    embed_time = end_time - start_time

    start_time = time.monotonic()
    x_test_recon = decoder(z_test)
    end_time = time.monotonic()
    print("seconds: ", end_time - start_time)
    recon_time = end_time - start_time
    reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
        "network",
        2,
        dataset,
        embed_time,
        recon_time,
        embed_time + recon_time,
        len(X_test_flat)
    ]

seconds:  0.05426294496282935
seconds:  0.0743955741636455
seconds:  0.13476407900452614
seconds:  0.002865093993023038
seconds:  0.19315936509519815
seconds:  0.0034132609143853188
seconds:  0.1907648160122335
seconds:  0.003230675123631954
seconds:  0.1927320130635053
seconds:  0.004346746951341629
seconds:  0.18912101699970663
seconds:  0.0031638320069760084
seconds:  0.1919563899282366
seconds:  0.004222973017022014
seconds:  0.18893632083199918
seconds:  0.003089851001277566
seconds:  0.1933124689385295
seconds:  0.0034024089109152555
seconds:  0.193087822990492
seconds:  0.002690219087526202



In [33]:
with tf.device('/CPU:0'):
    n_repeats = 10
    for i in tqdm(range(n_repeats)):
        start_time = time.monotonic()
        z_test = encoder(X_test)
        end_time = time.monotonic()
        print("seconds: ", end_time - start_time)
        embed_time = end_time - start_time

        start_time = time.monotonic()
        x_test_recon = decoder(z_test)
        end_time = time.monotonic()
        print("seconds: ", end_time - start_time)
        recon_time = end_time - start_time
        reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
            "network-cpu",
            2,
            dataset,
            embed_time,
            recon_time,
            embed_time + recon_time,
        len(X_test_flat)
        ]

seconds:  0.3598880870267749
seconds:  1.8515937111806124
seconds:  0.18340717116370797
seconds:  1.7501369169913232
seconds:  0.17800314212217927
seconds:  2.1637688749469817
seconds:  0.22455259296111763
seconds:  1.792737961979583
seconds:  0.19474253081716597
seconds:  1.7302633959334344
seconds:  0.18382644187659025
seconds:  1.7763448529876769
seconds:  0.23063940089195967
seconds:  1.7694010799750686
seconds:  0.1918868781067431
seconds:  1.7612372408621013
seconds:  0.18733517499640584
seconds:  1.8651142169255763
seconds:  0.1847074879333377
seconds:  1.8710570309776813



In [34]:
X_recon = decoder(encoder(X_test)).numpy()
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

In [35]:
MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

In [36]:
reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['network', 2, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,fmnist,0.024047,0.082691,0.035981,0.617169
1,AE,64,fmnist,0.009165,0.043426,0.013206,0.807800
2,network,2,fmnist,0.026576,0.092112,0.046975,0.598707


##### 64 dims

In [37]:
load_loc = output_dir / dataset / '64' / 'recon-network' 

In [38]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    decoding_method = "autoencoder",
    batch_size = 100,
    dims = dims
)

In [39]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [40]:
decoder = tf.keras.models.load_model((load_loc / 'decoder').as_posix())
embedder.decoder = decoder

In [41]:
n_repeats = 10
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    z_test = encoder(X_test)
    end_time = time.monotonic()
    print("seconds: ", end_time - start_time)
    embed_time = end_time - start_time

    start_time = time.monotonic()
    x_test_recon = decoder(z_test)
    end_time = time.monotonic()
    print("seconds: ", end_time - start_time)
    recon_time = end_time - start_time
    reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
        "network",
        64,
        dataset,
        embed_time,
        recon_time,
        embed_time + recon_time,
        len(X_test_flat)
    ]

seconds:  0.0584695590659976
seconds:  0.07318920898251235
seconds:  0.14903888502158225
seconds:  0.00425391411408782
seconds:  0.20253004785627127
seconds:  0.005087448982521892
seconds:  0.1990684769116342
seconds:  0.005126109113916755
seconds:  0.20293387002311647
seconds:  0.005266503896564245
seconds:  0.1994154080748558
seconds:  0.005365957040339708
seconds:  0.20066305389627814
seconds:  0.0054174079559743404
seconds:  0.1987611090298742
seconds:  0.005193942226469517
seconds:  0.2005142099224031
seconds:  0.004918833961710334
seconds:  0.19883662089705467
seconds:  0.004451749846339226



In [42]:
with tf.device("/CPU:0"):
    n_repeats = 10
    for i in tqdm(range(n_repeats)):
        start_time = time.monotonic()
        z_test = encoder(X_test)
        end_time = time.monotonic()
        print("seconds: ", end_time - start_time)
        embed_time = end_time - start_time

        start_time = time.monotonic()
        x_test_recon = decoder(z_test)
        end_time = time.monotonic()
        print("seconds: ", end_time - start_time)
        recon_time = end_time - start_time
        reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
            "network-cpu",
            64,
            dataset,
            embed_time,
            recon_time,
            embed_time + recon_time,
        len(X_test_flat)
        ]

seconds:  0.33381752599962056
seconds:  1.9603396700695157
seconds:  0.18107500299811363
seconds:  1.9230148589704186
seconds:  0.18332317797467113
seconds:  1.8596302170772105
seconds:  0.19082226580940187
seconds:  1.8471316120121628
seconds:  0.18786470009945333
seconds:  1.87449124106206
seconds:  0.1902146991342306
seconds:  1.8536553119774908
seconds:  0.17953033605590463
seconds:  1.8285825899802148
seconds:  0.18782190885394812
seconds:  1.9264386091381311
seconds:  0.17810207488946617
seconds:  1.8972847671248019
seconds:  0.18317439407110214
seconds:  1.840892189880833



In [43]:
reconstruction_speed_df

,method_,dimensions,dataset,embed_time,recon_time,speed,nex
0,network,2,fmnist,0.054263,0.074396,0.128659,10000
1,network,2,fmnist,0.134764,0.002865,0.137629,10000
2,network,2,fmnist,0.193159,0.003413,0.196573,10000
3,network,2,fmnist,0.190765,0.003231,0.193995,10000
4,network,2,fmnist,0.192732,0.004347,0.197079,10000
5,network,2,fmnist,0.189121,0.003164,0.192285,10000
6,network,2,fmnist,0.191956,0.004223,0.196179,10000
7,network,2,fmnist,0.188936,0.003090,0.192026,10000
8,network,2,fmnist,0.193312,0.003402,0.196715,10000
9,network,2,fmnist,0.193088,0.002690,0.195778,10000


In [44]:
X_recon = decoder(encoder(X_test)).numpy()
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

In [45]:
MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

In [46]:
reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['network', 64, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,fmnist,0.024047,0.082691,0.035981,0.617169
1,AE,64,fmnist,0.009165,0.043426,0.013206,0.807800
2,network,2,fmnist,0.026576,0.092112,0.046975,0.598707
3,network,64,fmnist,0.024062,0.087135,0.043900,0.624377


#### UMAP-learn

##### 2 dims

In [47]:
embedder = UMAP(n_components = 2, verbose=True)
z_umap = embedder.fit_transform(X_train_flat)

UMAP(dens_frac=0.0, dens_lambda=0.0, verbose=True)
Construct fuzzy simplicial set
Fri Jul 17 15:17:03 2020 Finding Nearest Neighbors
Fri Jul 17 15:17:03 2020 Building RP forest with 16 trees
Fri Jul 17 15:17:05 2020 parallel NN descent for 16 iterations
	 0  /  16
	 1  /  16
	 2  /  16
	 3  /  16
Fri Jul 17 15:17:15 2020 Finished Nearest Neighbor Search
Fri Jul 17 15:17:17 2020 Construct embedding
	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Fri Jul 17 15:17:47 2020 Finished embedding


In [48]:
x_test_samples= []
x_test_recon_samples= []
n_repeats = 10
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    z_test = embedder.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    embed_time = end_time - start_time

    nex = 10 # it would take far too long to reconstruct the entire dataset
    samp_idx = np.random.randint(len(z_test),  size= nex)
    sample = np.array(z_test)[samp_idx]
    x_test_samples.append(samp_idx)
    start_time = time.monotonic()
    x_test_recon = embedder.inverse_transform(sample);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    recon_time = (end_time - start_time)*len(z_test)/nex

    reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
        "umap-learn",
        2,
        dataset,
        embed_time,
        recon_time,
        embed_time + recon_time,
        len(X_test_flat)
    ]
    x_test_recon_samples.append(x_test_recon)

	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  17.304555150913075
	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  54.38560674805194
	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  4.860220918897539
	completed  0  /  

In [49]:
x_recon = np.concatenate(x_test_recon_samples)

In [50]:
x_real = np.array(X_test_flat)[np.concatenate(x_test_samples)]

In [51]:

MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['umap-learn', 2, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,fmnist,0.024047,0.082691,0.035981,0.617169
1,AE,64,fmnist,0.009165,0.043426,0.013206,0.807800
2,network,2,fmnist,0.026576,0.092112,0.046975,0.598707
3,network,64,fmnist,0.024062,0.087135,0.043900,0.624377
4,umap-learn,2,fmnist,0.033949,0.114089,0.067095,-16.062110


##### PCA

##### 2 dims


In [52]:
pca = PCA(n_components=2)
z = pca.fit_transform(X_train_flat)

In [53]:
n_repeats = 10
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    z_test = pca.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    embed_time = end_time - start_time

    start_time = time.monotonic()
    x_test_recon = pca.inverse_transform(z_test);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    recon_time = (end_time - start_time)

    reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
        "pca",
        2,
        dataset,
        embed_time,
        recon_time,
        embed_time + recon_time,
        len(X_test_flat)
    ]

seconds:  0.011283128056675196
seconds:  0.019574179081246257
seconds:  0.010463193990290165
seconds:  0.015611073933541775
seconds:  0.01090701692737639
seconds:  0.014593943953514099
seconds:  0.009654992027208209
seconds:  0.012596486136317253
seconds:  0.009446454932913184
seconds:  0.013155843131244183
seconds:  0.009587208973243833
seconds:  0.012656678212806582
seconds:  0.009444844909012318
seconds:  0.012651188066229224
seconds:  0.009678282076492906
seconds:  0.012468982953578234
seconds:  0.010335741098970175
seconds:  0.013419230002909899
seconds:  0.009495676029473543
seconds:  0.012501433957368135



In [54]:
X_recon = pca.inverse_transform(pca.transform(X_test_flat))
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['pca', 2, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,fmnist,0.024047,0.082691,0.035981,0.617169
1,AE,64,fmnist,0.009165,0.043426,0.013206,0.807800
2,network,2,fmnist,0.026576,0.092112,0.046975,0.598707
3,network,64,fmnist,0.024062,0.087135,0.043900,0.624377
4,umap-learn,2,fmnist,0.033949,0.114089,0.067095,-16.062110
5,pca,2,fmnist,0.046097,0.150541,0.117590,0.361385


##### 64 dims

In [55]:
pca = PCA(n_components=64)
z = pca.fit_transform(X_train_flat)

In [56]:
n_repeats = 10
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    z_test = pca.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    embed_time = end_time - start_time

    start_time = time.monotonic()
    x_test_recon = pca.inverse_transform(z_test);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    recon_time = (end_time - start_time)

    reconstruction_speed_df.loc[len(reconstruction_speed_df)] = [
        "pca",
        64,
        dataset,
        embed_time,
        recon_time,
        embed_time + recon_time,
        len(X_test_flat)
    ]

seconds:  0.01983020710758865
seconds:  0.02135446108877659
seconds:  0.018131237011402845
seconds:  0.016668994911015034
seconds:  0.014909004094079137
seconds:  0.01711346791125834
seconds:  0.014495291979983449
seconds:  0.01613001897931099
seconds:  0.014859982067719102
seconds:  0.017564551206305623
seconds:  0.014653336023911834
seconds:  0.016068938188254833
seconds:  0.014816680923104286
seconds:  0.017417626921087503
seconds:  0.01451661204919219
seconds:  0.016055436804890633
seconds:  0.01486961217597127
seconds:  0.016991224139928818
seconds:  0.014543203171342611
seconds:  0.016278042923659086



In [57]:
X_recon = pca.inverse_transform(pca.transform(X_test_flat))
x_real = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))
x_recon = X_recon.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

MSE = mean_squared_error(
    x_real, 
    x_recon
)
MAE = mean_absolute_error(
    x_real, 
    x_recon
)
MedAE = median_absolute_error(
    x_real, 
    x_recon
)
R2 = r2_score(
    x_real, 
    x_recon
)

reconstruction_acc_df.loc[len(reconstruction_acc_df)] = ['pca', 64, dataset, MSE, MAE, MedAE, R2]
reconstruction_acc_df

,method_,dimensions,dataset,MSE,MAE,MedAE,R2
0,AE,2,fmnist,0.024047,0.082691,0.035981,0.617169
1,AE,64,fmnist,0.009165,0.043426,0.013206,0.807800
2,network,2,fmnist,0.026576,0.092112,0.046975,0.598707
3,network,64,fmnist,0.024062,0.087135,0.043900,0.624377
4,umap-learn,2,fmnist,0.033949,0.114089,0.067095,-16.062110
5,pca,2,fmnist,0.046097,0.150541,0.117590,0.361385
6,pca,64,fmnist,0.010374,0.063742,0.039000,0.774954


### Save

In [58]:
save_loc = DATA_DIR / 'reconstruction_speed' / (dataset + '.pickle')
ensure_dir(save_loc)
reconstruction_speed_df.to_pickle(save_loc)

In [59]:
save_loc = DATA_DIR / 'reconstruction_acc' / (dataset + '.pickle')
ensure_dir(save_loc)
reconstruction_acc_df.to_pickle(save_loc)